In [3]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
import csv 
import matplotlib.pyplot as plt 
plt.style.use('seaborn')
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
from matplotlib.ticker import FuncFormatter
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [5]:
df = pd.read_excel("Data_DAP.xlsx")

print("Original Data:")
print(df.head())

Original Data:
                                              title         price  timeStamp  \
0  XM MINH HUY- Hỗ Trợ Trả Góp Xe Đã Qua Sử Dụng  68.686.868 đ        NaN   
1      XE MÁY TÚ TÀI- CHUYÊN CÁC DÒNG HONDA CAO CẤP  68.686.868 đ        NaN   
2             Sh CBS 150cc 2018 biển số TP.Nam Định  62.999.999 đ        NaN   
3                      Xe LEAD nguyên bản, như hình   6.800.000 đ        NaN   
4       Honda Air Blade 125. Bản Sporrt. Chính chủ.  15.600.000 đ        NaN   

    hang       dong  namdangki   soKM   tinhTrang  loaiXe      dungTich  \
0  Honda         SH       2022   8000  Đã sử dụng  Tay ga           NaN   
1  Honda         SH       2022   1000  Đã sử dụng  Tay ga  100 - 175 cc   
2  Honda         SH       2018   8900  Đã sử dụng  Tay ga  100 - 175 cc   
3  Honda       Lead       2012  16800  Đã sử dụng  Tay ga  100 - 175 cc   
4  Honda  Air Blade       2016  18600  Đã sử dụng  Tay ga  100 - 175 cc   

   xuatXu    chinhSachBH TrongLuong  
0     NaN  Bảo 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3358 entries, 0 to 3357
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        3358 non-null   object 
 1   price        3358 non-null   object 
 2   timeStamp    0 non-null      float64
 3   hang         3358 non-null   object 
 4   dong         3358 non-null   object 
 5   namdangki    3358 non-null   int64  
 6   soKM         3358 non-null   int64  
 7   tinhTrang    3358 non-null   object 
 8   loaiXe       3358 non-null   object 
 9   dungTich     2944 non-null   object 
 10  xuatXu       0 non-null      float64
 11  chinhSachBH  3358 non-null   object 
 12  TrongLuong   3358 non-null   object 
dtypes: float64(2), int64(2), object(9)
memory usage: 341.2+ KB


In [7]:
print('Rows     :',df.shape[0])
print('Columns  :',df.shape[1])
print('\nFeatures :\n     :',df.columns.tolist())
print('\nMissing values    :',df.isnull().values.sum())
print('\nUnique values :  \n',df.nunique())

Rows     : 3358
Columns  : 13

Features :
     : ['title', 'price', 'timeStamp', 'hang', 'dong', 'namdangki', 'soKM', 'tinhTrang', 'loaiXe', 'dungTich', 'xuatXu', 'chinhSachBH', 'TrongLuong']

Missing values    : 7130

Unique values :  
 title          2938
price           449
timeStamp         0
hang             25
dong            125
namdangki        38
soKM            432
tinhTrang         1
loaiXe            3
dungTich          5
xuatXu            0
chinhSachBH       1
TrongLuong        1
dtype: int64


1. Data Processing

In [8]:
df['price'] = df['price'].str.replace('.', '').str.replace('đ', '')
df.drop(columns=['timeStamp'], inplace=True)
df.drop(columns=['xuatXu'], inplace=True)
df.drop(columns=['title'], inplace=True)
df.drop(columns=['TrongLuong'], inplace=True)

2. Mising Values

In [9]:
total = df.isnull().sum().sort_values(ascending=False)
percent = ((df.isnull().sum())*100)/df.isnull().count().sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total','Percent'], 
                         sort=False).sort_values('Total', ascending=False)
missing_data.head(40)

,Total,Percent
dungTich,414,12.328767
price,0,0.000000
hang,0,0.000000
dong,0,0.000000
namdangki,0,0.000000
soKM,0,0.000000
tinhTrang,0,0.000000
loaiXe,0,0.000000
chinhSachBH,0,0.000000


In [10]:
df['dungTich'].fillna('?', inplace=True)


filtered_data = df[df['dungTich'].str.count('\?') > 0]['dong']

# In ra các dòng duy nhất trong cột 'dong'
unique_dongs_with_question_mark = filtered_data.unique()

# In ra tên các dòng unique trong cột 'dong' có kí tự '?' ở cột 'dungTich' tương ứng
for dong in unique_dongs_with_question_mark:
    print("Row:", dong, "- have missing value:", 
          filtered_data[filtered_data == dong].count())

Row: SH - have missing value: 10
Row: Vision - have missing value: 46
Row: Exciter - have missing value: 68
Row: Winner X - have missing value: 33
Row: Wave - have missing value: 58
Row: Winner - have missing value: 19
Row: Lead - have missing value: 15
Row: Liberty - have missing value: 14
Row: Air Blade - have missing value: 49
Row: Nouvo - have missing value: 11
Row: Vespa - have missing value: 54
Row: Dream - have missing value: 15
Row: R - have missing value: 12
Row: Dòng khác - have missing value: 9
Row: Blade - have missing value: 1


2.1. Handeling Missing Value

In [11]:
dong_list = ['SH', 'Vision', 'Exciter', 'Winner X', 'Wave', 'Winner', 'Lead', 'Liberty', 
             'Air Blade', 'Nouvo', 'Vespa', 'Dream', 'R', 'Dòng khác', 'Blade']

# Lặp qua từng dòng và in ra các giá trị tần suất của cột 'dungTich'
for dong in dong_list:
    # Lọc dữ liệu cho dòng hiện tại
    a = df[df['dong'] == dong]
    
    # In ra thông tin số lượng giá trị trong cột 'dungTich'
    print("Row:", dong, "- have missing value:", 
          filtered_data[filtered_data == dong].count())
    print(a['dungTich'].value_counts())
    print()

Row: SH - have missing value: 10
100 - 175 cc    258
?                10
Trên 175 cc       3
50 - 100 cc       2
Name: dungTich, dtype: int64

Row: Vision - have missing value: 46
100 - 175 cc    196
?                46
50 - 100 cc       3
Trên 175 cc       1
Name: dungTich, dtype: int64

Row: Exciter - have missing value: 68
100 - 175 cc    157
?                68
Name: dungTich, dtype: int64

Row: Winner X - have missing value: 33
100 - 175 cc    47
?               33
50 - 100 cc      1
Trên 175 cc      1
Name: dungTich, dtype: int64

Row: Wave - have missing value: 58
100 - 175 cc     265
?                 58
50 - 100 cc       27
Dưới 50 cc         2
Không biết rõ      1
Name: dungTich, dtype: int64

Row: Winner - have missing value: 19
100 - 175 cc    35
?               19
Trên 175 cc      1
Name: dungTich, dtype: int64

Row: Lead - have missing value: 15
100 - 175 cc    151
?                15
50 - 100 cc       1
Name: dungTich, dtype: int64

Row: Liberty - have missing value: 14


In [12]:
df['price'] = df['price'].astype(float)
df['price'] = df['price'].astype(int)
df = df[(df['dungTich'] != 'Không biết rõ') & (df['dong'] != 'Dòng khác')]

In [13]:
for dong in dong_list:
    # Lọc dữ liệu cho dòng hiện tại
    a = df[df['dong'] == dong]
    
    # Tìm giá trị cao nhất trong cột 'dungTich'
    max_value = df['dungTich'].mode().values[0]
    
    # Thay thế giá trị '?' trong cột 'dungTich' bằng giá trị cao nhất
    df.loc[(df['dong'] == dong) & (df['dungTich'] == '?'), 'dungTich'] = max_value
print(df.head(5
))

      price   hang       dong  namdangki   soKM   tinhTrang  loaiXe  \
0  68686868  Honda         SH       2022   8000  Đã sử dụng  Tay ga   
1  68686868  Honda         SH       2022   1000  Đã sử dụng  Tay ga   
2  62999999  Honda         SH       2018   8900  Đã sử dụng  Tay ga   
3   6800000  Honda       Lead       2012  16800  Đã sử dụng  Tay ga   
4  15600000  Honda  Air Blade       2016  18600  Đã sử dụng  Tay ga   

       dungTich    chinhSachBH  
0  100 - 175 cc  Bảo hành hãng  
1  100 - 175 cc  Bảo hành hãng  
2  100 - 175 cc  Bảo hành hãng  
3  100 - 175 cc  Bảo hành hãng  
4  100 - 175 cc  Bảo hành hãng  


3. Visualize Data

In [14]:
print('Rows     :',df.shape[0])
print('Columns  :',df.shape[1])
print('\nFeatures :\n     :',df.columns.tolist())
print('\nMissing values    :',df.isnull().values.sum())
print('\nUnique values :  \n',df.nunique())
data= df

Rows     : 3272
Columns  : 9

Features :
     : ['price', 'hang', 'dong', 'namdangki', 'soKM', 'tinhTrang', 'loaiXe', 'dungTich', 'chinhSachBH']

Missing values    : 0

Unique values :  
 price          440
hang            23
dong           124
namdangki       38
soKM           422
tinhTrang        1
loaiXe           3
dungTich         4
chinhSachBH      1
dtype: int64


In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,3272.0,2.349749e+07,4.876352e+07,-2.147484e+09,9000000.0,16500000.0,26800000.0,455000000.0
namdangki,3272.0,2.017329e+03,5.185124e+00,1.981000e+03,2015.0,2018.0,2021.0,2024.0
soKM,3272.0,1.608145e+04,5.650099e+04,1.000000e+00,2000.0,5999.0,12000.0,986422.0


Some important observations from the dataset:

Lowest namdangki is 1981, highest is 2024

Highest soKM is 986,422

Price is 23,919,510 vnd